In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import time, os
import torch
from torch.utils import data

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import roc_auc_score

RANDOM_STATE = 42
N_KEYWORDS= 500
N_AUTHORS = 2302

In [2]:
train_df = pd.read_json('../data/train.json', orient='index')

ValueError: Expected object or value

In [ ]:
def preprocess_dataset(df, is_test_data= False):
    if is_test_data:
        pass
    else:
        df1 = pd.DataFrame(df.author)
        df1.columns=['coauthor']
        df2 = df.explode('author')
        df3 = df1.join(df2)
        df3['author_set'] = df3['author'].apply(lambda x: {x})
        df3['coauthor_set'] = df3['coauthor'].apply(lambda x: set(x))
        df3['coauthor'] = df3['coauthor_set'] - df3['author_set']
        df3['coauthor'] = df3['coauthor'].apply(lambda x: list(x))
        df3 = df3[['keywords', 'coauthor', 'author']]
        mlb_keywords = MultiLabelBinarizer(sparse_output=True, classes=list(range(500)))
        keywords_oh = mlb_keywords.fit_transform(df3.keywords).toarray()

        mlb_coauthor = MultiLabelBinarizer(sparse_output=True, classes=list(range(2302)))
        coauthor_oh = mlb_coauthor.fit_transform(df3.coauthor).toarray()

        features = np.hstack((keywords_oh, coauthor_oh))
        labels = df3.author.to_list()
    
    return features, labels, mlb_keywords, mlb_coauthor

In [ ]:
features, labels, mlb_keywords, mlb_coauthor = preprocess_dataset(train_df)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)

In [ ]:
roc_auc_score(y_val, clf.predict_proba(X_val), multi_class='ovo')